In [ ]:
import importlib
from pymongo import MongoClient
from datetime import datetime, timedelta
import sys
import eduhub_queries
from pprint import pprint
importlib.reload(eduhub_queries)
from eduhub_queries import *
from pprint import pprint


In [35]:
client = MongoClient("mongodb://localhost:27017/")
db = client["eduhub_db"]


# **TASK 3**

## **Task 3.1: Create Operations**

**Inserting to the users collection**

In [42]:
new_student = {
    "_id": "user31",
    "email": "beckynd@gmail.com",
    "firstName": "Becky",
    "lastName": "Ndubuisi",
    "role": "student",
    "dateJoined": datetime.utcnow(),
    "profile": {
        "bio": "Excited to learn!",
        "avatar": "https://example.com/avatar.png",
        "skills": ["Python", "MongoDB"]
    },
    "isActive": False
}

In [43]:
insert_one('users',new_student)

✅ Inserted into 'users' with _id: user31


'user31'

**Inserting to the courses collection**

In [19]:
instructor_user = db.users.find_one({"role": "instructor"})
instructor_userId = instructor_user["_id"]

new_course = {
    "_id": "course9",
    "title": "MongoDB for Beginners",
    "description": "Learn the basics of MongoDB and NoSQL design.",
    "instructorId": instructor_userId,
    "category": "Databases",
    "level": "beginner",
    "duration": 10,  # in hours
    "price": 49.99,
    "tags": ["mongodb", "nosql", "database"],
    "createdAt": datetime.utcnow(),
    "updatedAt": datetime.utcnow(),
    "isPublished": False
}


insert_one('courses',new_course)

✅ Inserted into 'courses' with _id: course9


'course9'

**Inserting into the enrollments collection**

In [21]:
student_user = db.users.find_one({"role": "student"})
studentId = student_user["_id"]

course = db.courses.find_one()
courseId = course["_id"]

enrollment = {
    "_id": "enroll26",
    "studentId": studentId,
    "courseId": courseId,
    "enrolledAt": datetime.utcnow(),
    "status": "in progress"
}

insert_one('enrollments',enrollment)

✅ Inserted into 'enrollments' with _id: enroll26


'enroll26'

**Inserting into lessons collection**

In [22]:
course = db.courses.find_one()
courseId = course["_id"]

lesson = {
    "_id": f"lesson_{courseId}_4",
    "courseId": courseId,
    "title": "Introduction to MongoDB",
    "content": "This lesson introduces MongoDB fundamentals.",
    "duration": 10,
    "order": 4,
    "resources": ["https://docs.mongodb.com"],
    "createdAt": datetime.utcnow(),
    "updatedAt": datetime.utcnow()
}

insert_one('lessons',lesson)

✅ Inserted into 'lessons' with _id: lesson_course1_4


'lesson_course1_4'

## **Task 3.2: Read Operations**

**Find all Active Student**

In [44]:
students = find_active_students(db)
for student in students:
    print(student["firstName"], student["lastName"])

Christopher Davis
George Torres
Brittany Smith
Brenda Dominguez
Eric Hall
Jennifer Shaffer
Megan Mcdaniel
David Burton
James Mason
Lori Reed
David Simpson
Colleen Hudson
Kendra Morton
Susan Rodriguez
Raymond Erickson
Samantha Harris
Jerry Williams
Austin Delacruz
Joseph Murray
Derrick Clark
John Conner
Amanda Ramirez
Emily Peterson
Jose Smith


**Retrieve course details with instructor information**

In [46]:
courses = get_courses_with_instructors(db)
for c in courses:
    instructor = c.get("instructor", {})
    print(f"{c['title']} - {instructor.get('firstName', 'N/A')} {instructor.get('lastName', '')}")


Program Masterclass - Lisa Valdez
Politics Masterclass - Billy Green
Step Masterclass - Raymond Newman
Mr Masterclass - Rachel Miles
Star Masterclass - Billy Green
Officer Masterclass - Lisa Valdez
Art Masterclass - Frank Bowen
Condition Masterclass - Billy Green
MongoDB for Beginners - Billy Green


**Get all courses in a specific category**

In [48]:
ds_courses = get_courses_by_category(db, "Databases")
for course in ds_courses:
    print(course["title"])

MongoDB for Beginners


**Find students enrolled in a particular course**

In [68]:
course_title, students = get_students_in_course(db, "course1")
if course_title:
    for student in students:
        print(f"{student['firstName']} {student['lastName']} ({student['_id']}) - {course_title}")
else:
    print("Course not found.")


David Simpson (user11) - Program Masterclass
Susan Rodriguez (user14) - Program Masterclass
Christopher Davis (user1) - Program Masterclass


**Search courses by title (case-insensitive, partial match)**

In [70]:
matched_courses = search_courses_by_title(db, "mongodb")
for course in matched_courses:
    print(course["title"])


MongoDB for Beginners


# **Task 3.3: Update Operations**

**Update a user’s profile information**

In [76]:
profile_data = {
    "bio": "Experienced data scientist and Python developer.",
    "avatar": "http://example.com/avatar.jpg",
    "skills": ["Python", "Data Science", "MongoDB"]
}


update_document(
    db,
    "users",
    {"_id": "user12"},
    {"$set": {"profile": profile_data}}
)

db.users.find_one({'_id':"user12"})

{'_id': 'user12',
 'email': 'xharrell@example.com',
 'firstName': 'Colleen',
 'lastName': 'Hudson',
 'role': 'student',
 'dateJoined': datetime.datetime(2025, 3, 20, 12, 57, 56, 991000),
 'profile': {'bio': 'Experienced data scientist and Python developer.',
  'avatar': 'http://example.com/avatar.jpg',
  'skills': ['Python', 'Data Science', 'MongoDB']},
 'isActive': True}

**Mark a course as published**

In [83]:
update_document(
    db,
    "courses",
    {"_id": "course8"},
    {"$set": {"isPublished": True}}
)


db.courses.find_one({'_id':"course8"})

{'_id': 'course8',
 'title': 'Condition Masterclass',
 'description': 'Require perform beautiful institution imagine. Two education career discussion reflect. Yet economic other hair reach.',
 'instructorId': 'user25',
 'category': 'AI',
 'level': 'beginner',
 'duration': 49,
 'price': 115.6,
 'tags': ['body', 'east', 'time', 'where'],
 'createdAt': datetime.datetime(2025, 6, 14, 13, 9, 17, 918000),
 'updatedAt': datetime.datetime(2025, 6, 14, 13, 9, 17, 918000),
 'isPublished': True}

**Update assignment grades**

In [81]:
update_document(
    db,
    "assignments",
    {"_id": "assign_course5_2"},
    {"$set": {"grade": 95.5}}
)

db.assignments.find_one({'_id':"assign_course5_2"})

{'_id': 'assign_course5_2',
 'courseId': 'course5',
 'lessonId': None,
 'title': 'Assignment for Star Masterclass',
 'description': 'Join rate conference score only under her. Game officer of free. Building dark edge sign moment alone. Same ok husband.',
 'createdAt': datetime.datetime(2025, 6, 14, 13, 13, 36, 363000),
 'dueDate': datetime.datetime(2025, 6, 19, 13, 13, 36, 363000),
 'attachmentUrl': 'https://woodward.com/',
 'grade': 95.5}

**Add tags to an existing course**

In [82]:
update_document(
    db,
    "courses",
    {"_id": "course9"},
    {"$addToSet": {"tags": {"$each": ["shell", "python"]}}}
)

db.courses.find_one({'_id':"course9"})


{'_id': 'course9',
 'title': 'MongoDB for Beginners',
 'description': 'Learn the basics of MongoDB and NoSQL design.',
 'instructorId': 'user25',
 'category': 'Databases',
 'level': 'beginner',
 'duration': 10,
 'price': 49.99,
 'tags': ['mongodb', 'nosql', 'database', 'shell', 'python'],
 'createdAt': datetime.datetime(2025, 6, 14, 13, 38, 57, 567000),
 'updatedAt': datetime.datetime(2025, 6, 14, 13, 38, 57, 567000),
 'isPublished': False}

## **Task 3.4: Delete Operations**

**Remove a user (soft delete by setting isActive to false)**

In [86]:
soft_delete_user(db, "user20")


db.users.find_one({'_id':"user20"})

{'_id': 'user20',
 'email': 'lauren22@example.org',
 'firstName': 'Derrick',
 'lastName': 'Clark',
 'role': 'student',
 'dateJoined': datetime.datetime(2025, 3, 27, 7, 34, 3, 176000),
 'profile': {'bio': 'Sound nothing black seek buy range.',
  'avatar': 'https://placekitten.com/432/408',
  'skills': ['front', 'why', 'current']},
 'isActive': False}

**Delete an enrollment**

In [90]:
result = delete_document(db, "enrollments", {"_id": "enroll15"})
print(f"Deleted {result['deleted_count']} enrollment(s).")


db.enrollments.find_one({'_id':"enroll15"})

Deleted 1 enrollment(s).


**Remove a lesson from a course**

In [93]:
delete_document(db, "lessons", {"_id": "lesson_course1_2"})
print(f"Deleted {result['deleted_count']} lesson(s).")

db.lessons.find_one({'_id':"lesson_course1_2"})

Deleted 1 lesson(s).


# **Part 4: Advanced Queries and Aggregation**

## **Task 4.1: Complex Queries**

**Find courses with price between $50 and $200**

In [108]:
courses = find_courses_in_price_range(db, 50, 200)
titles = [c["title"] for c in courses]
print("Courses $50-$200:")
pprint(titles)

Courses $50-$200:
['Program Masterclass',
 'Politics Masterclass',
 'Step Masterclass',
 'Mr Masterclass',
 'Star Masterclass',
 'Officer Masterclass',
 'Art Masterclass',
 'Condition Masterclass']


**Get users who joined in the last 6 months**

In [109]:
users = find_recent_users(db, 6)
print("Recent users:")
pprint([u["email"] for u in users])

Recent users:
['kelsey22@example.net',
 'pwhitaker@example.org',
 'dianafuentes@example.net',
 'brendagonzalez@example.com',
 'alexanderhernandez@example.net',
 'sarah23@example.org',
 'burgessrhonda@example.org',
 'kayla20@example.net',
 'donaldday@example.net',
 'lucasadam@example.com',
 'asmith@example.com',
 'xharrell@example.com',
 'kmcgee@example.org',
 'xrodriguez@example.com',
 'justin31@example.org',
 'usmith@example.org',
 'gonzalezjessica@example.com',
 'gutierrezalec@example.org',
 'zberg@example.com',
 'lauren22@example.org',
 'oking@example.net',
 'ilane@example.org',
 'danielpeterson@example.net',
 'lynnclifford@example.net',
 'carlsonlindsey@example.net',
 'llopez@example.com',
 'lisa78@example.org',
 'zhinton@example.com',
 'paul15@example.com',
 'townsendryan@example.com',
 'beckynd@gmail.com']


**Find courses that have specific tags using $in operator**

In [103]:
courses_with_tags = find_courses_by_tags(db, ["python", "shell"])
print("Courses with tags:", [c["title"] for c in courses_with_tags])

Courses with tags: ['MongoDB for Beginners']


**Retrieve assignments with due dates in the next week**

In [110]:
from pprint import pprint

assignments_due = find_assignments_due_next_week(db)
print("Assignments due next week:")
pprint([a["title"] for a in assignments_due])


Assignments due next week:
['Assignment for Program Masterclass',
 'Assignment for Mr Masterclass',
 'Assignment for Mr Masterclass',
 'Assignment for Star Masterclass',
 'Assignment for Star Masterclass',
 'Assignment for Art Masterclass',
 'Assignment for Condition Masterclass']


## **Task 4.2: Aggregation Pipeline**

**Count total enrollments per course**

In [112]:
print("Enrollment Stats:")
for e in get_enrollment_stats(db):
    print(e)

Enrollment Stats:
{'totalEnrollments': 3, 'courseId': 'course8', 'courseTitle': 'Condition Masterclass'}
{'totalEnrollments': 4, 'courseId': 'course7', 'courseTitle': 'Art Masterclass'}
{'totalEnrollments': 3, 'courseId': 'course4', 'courseTitle': 'Mr Masterclass'}
{'totalEnrollments': 3, 'courseId': 'course2', 'courseTitle': 'Politics Masterclass'}
{'totalEnrollments': 6, 'courseId': 'course3', 'courseTitle': 'Step Masterclass'}
{'totalEnrollments': 3, 'courseId': 'course6', 'courseTitle': 'Officer Masterclass'}
{'totalEnrollments': 3, 'courseId': 'course1', 'courseTitle': 'Program Masterclass'}


**Calculate average course rating**

In [119]:
print("\nAverage Course Rating:")
print(get_avg_course_rating(db))


Average Course Rating:
3.755555555555555


**Group by course category**

In [114]:
print("\nCourses Grouped by Category:")
for c in group_courses_by_category(db):
    print(c)


Courses Grouped by Category:
{'count': 1, 'courses': ['Star Masterclass'], 'category': 'Marketing'}
{'count': 1, 'courses': ['MongoDB for Beginners'], 'category': 'Databases'}
{'count': 3, 'courses': ['Program Masterclass', 'Step Masterclass', 'Mr Masterclass'], 'category': 'Web Dev'}
{'count': 2, 'courses': ['Officer Masterclass', 'Condition Masterclass'], 'category': 'AI'}
{'count': 2, 'courses': ['Politics Masterclass', 'Art Masterclass'], 'category': 'Design'}


## **Student Performance Analysis**

**Average grade per student**

In [136]:
print("Average Grade per Student:")
for grade in get_avg_grade_per_student(db):
    print(grade)

Average Grade per Student:
{'studentId': 'user9', 'firstName': 'James', 'lastName': 'Mason', 'averageGrade': 95.83}
{'studentId': 'user18', 'firstName': 'Austin', 'lastName': 'Delacruz', 'averageGrade': 83.06}
{'studentId': 'user2', 'firstName': 'George', 'lastName': 'Torres', 'averageGrade': 81.52}
{'studentId': 'user11', 'firstName': 'David', 'lastName': 'Simpson', 'averageGrade': 75.76}
{'studentId': 'user5', 'firstName': 'Eric', 'lastName': 'Hall', 'averageGrade': 74.55}
{'studentId': 'user3', 'firstName': 'Brittany', 'lastName': 'Smith', 'averageGrade': 72.6}
{'studentId': 'user23', 'firstName': 'Emily', 'lastName': 'Peterson', 'averageGrade': 70.53}


**Completion rate by course**

In [127]:
print("\nCompletion Rate by Course:")
for complete in get_completion_rate_by_course(db):
    print(complete)


Completion Rate by Course:
{'courseId': 'course7', 'enrolled': 4, 'submitted': 2, 'completionRate': 50.0}
{'courseId': 'course4', 'enrolled': 3, 'submitted': 2, 'completionRate': 66.67}
{'courseId': 'course2', 'enrolled': 3, 'submitted': 6, 'completionRate': 200.0}
{'courseId': 'course3', 'enrolled': 6, 'submitted': 2, 'completionRate': 33.33}
{'courseId': 'course6', 'enrolled': 3, 'submitted': 1, 'completionRate': 33.33}
{'courseId': 'course1', 'enrolled': 3, 'submitted': 3, 'completionRate': 100.0}
{'courseId': 'course8', 'enrolled': 3, 'submitted': 2, 'completionRate': 66.67}


**Top-performing students**

In [135]:
print("\nTop Performing Students:")
for perform in get_top_performing_students(db,5):
    print(perform)


Top Performing Students:
{'_id': 'user9', 'submissionCount': 1, 'studentId': 'user9', 'averageGrade': 95.83, 'name': 'James Mason'}
{'_id': 'user18', 'submissionCount': 3, 'studentId': 'user18', 'averageGrade': 83.06, 'name': 'Austin Delacruz'}
{'_id': 'user2', 'submissionCount': 1, 'studentId': 'user2', 'averageGrade': 81.52, 'name': 'George Torres'}
{'_id': 'user11', 'submissionCount': 2, 'studentId': 'user11', 'averageGrade': 75.76, 'name': 'David Simpson'}
{'_id': 'user5', 'submissionCount': 1, 'studentId': 'user5', 'averageGrade': 74.55, 'name': 'Eric Hall'}


## **Instructor Analytics**


**Total students taught by each instructor**

In [131]:
students_per_instructor = get_total_students_per_instructor(db)
print("\nTotal Students per Instructor:")
for item in students_per_instructor:
    print(f"Instructor ID: {item['instructorId']}, Total Students: {item['totalStudents']}")



Total Students per Instructor:
Instructor ID: user26, Total Students: 6
Instructor ID: user28, Total Students: 3
Instructor ID: user25, Total Students: 6
Instructor ID: user29, Total Students: 5
Instructor ID: user27, Total Students: 4


**Average course rating per instructor**


In [132]:
avg_rating = get_avg_rating_per_instructor(db)
print("\nAverage Course Rating per Instructor:")
for item in avg_rating:
    print(f"Instructor ID: {item['instructorId']}, Average Rating: {round(item['avgRating'], 2)}")



Average Course Rating per Instructor:
Instructor ID: user28, Average Rating: 4.0
Instructor ID: user29, Average Rating: 3.0
Instructor ID: user26, Average Rating: 4.5
Instructor ID: user27, Average Rating: 2.5
Instructor ID: user25, Average Rating: 3.83



**Revenue generated per instructor**


In [133]:
revenue_data = get_revenue_per_instructor(db)
print("\nRevenue per Instructor:")
for item in revenue_data:
    print(f"Instructor ID: {item['instructorId']}, Revenue: ${item['revenue']:.2f}")



Revenue per Instructor:
Instructor ID: user26, Revenue: $571.53
Instructor ID: user28, Revenue: $472.62
Instructor ID: user25, Revenue: $618.36
Instructor ID: user29, Revenue: $364.62
Instructor ID: user27, Revenue: $557.28


## **Advanced Analytics**


**Monthly enrollment trends**


In [138]:
# Monthly enrollment trends for last 6 months
monthly_trends = get_monthly_enrollment_trends(db,6)
print("Monthly Enrollment Trends:")
for item in monthly_trends:
    print(f"{item['year']}-{item['month']:02d}: {item['enrollments']} enrollments")


Monthly Enrollment Trends:
2025-01: 10 enrollments
2025-02: 3 enrollments
2025-03: 5 enrollments
2025-04: 1 enrollments
2025-05: 4 enrollments
2025-06: 2 enrollments


**Most popular course categories**


In [139]:
popular_categories = get_most_popular_categories(db)
print("\nMost Popular Course Categories:")
for cat in popular_categories:
    print(f"{cat['_id']}: {cat['enrollmentCount']} enrollments")



Most Popular Course Categories:
Web Dev: 12 enrollments
Design: 7 enrollments
AI: 6 enrollments


**Student engagement metrics**

In [140]:
engagement_metrics = get_student_engagement_metrics(db)
print("\nStudent Engagement Metrics:")
for student in engagement_metrics:
    print(f"{student['name']} ({student['studentId']}): {student['assignmentsSubmitted']} submissions, Average Grade: {student['averageGrade']}")



Student Engagement Metrics:
George Torres (user2): 1 submissions, Average Grade: 81.52
Brittany Smith (user3): 1 submissions, Average Grade: 72.6
Eric Hall (user5): 1 submissions, Average Grade: 74.55
Austin Delacruz (user18): 3 submissions, Average Grade: 83.06
Emily Peterson (user23): 1 submissions, Average Grade: 70.53
Megan Mcdaniel (user7): 1 submissions, Average Grade: None
Kendra Morton (user13): 1 submissions, Average Grade: None
John Conner (user21): 1 submissions, Average Grade: None
David Simpson (user11): 2 submissions, Average Grade: 75.76
Brenda Dominguez (user4): 1 submissions, Average Grade: None
Samantha Harris (user16): 3 submissions, Average Grade: None
Susan Rodriguez (user14): 1 submissions, Average Grade: None
James Mason (user9): 2 submissions, Average Grade: 95.83
Joseph Murray (user19): 1 submissions, Average Grade: None


# **Part 5: Indexing and Performance**

## **Task 5.1: Index Creation**

In [141]:
# 1. User email lookup (unique index on email)
db.users.create_index("email", unique=True)

'email_1'

In [142]:
# 2. Course search by title and category (compound text index for efficient text search)
db.courses.create_index([
    ("title", "text"),
    ("category", "text")
])

'title_text_category_text'

In [143]:
# 3. Assignment queries by due date (index on dueDate)
db.assignments.create_index("dueDate")

'dueDate_1'

In [144]:
# 4. Enrollment queries by student and course (compound index on studentId and courseId)
db.enrollments.create_index([("studentId", 1), ("courseId", 1)])

'studentId_1_courseId_1'

## **Task 5.2: Query Optimization**

**Analyze query performance using explain() method in PyMongo**

In [152]:
explain_res = analyze_query_performance(db, "courses", {"category": "Programming"})


{'executionStages': {'advanced': 0,
                     'direction': 'forward',
                     'docsExamined': 9,
                     'executionTimeMillisEstimate': 0,
                     'filter': {'category': {'$eq': 'Programming'}},
                     'isCached': False,
                     'isEOF': 1,
                     'nReturned': 0,
                     'needTime': 9,
                     'needYield': 0,
                     'restoreState': 0,
                     'saveState': 0,
                     'stage': 'COLLSCAN',
                     'works': 10},
 'executionSuccess': True,
 'executionTimeMillis': 1,
 'nReturned': 0,
 'totalDocsExamined': 9,
 'totalKeysExamined': 0}


**Optimize at least 3 slow queries**

**Query Before index**

In [ ]:
# 1. Course by category
q1_before = time_query(db, "courses", {"category": "Programming"}, label="Course by Category")

# 2. Enrollment by student and course
q2_before = time_query(db, "enrollments", {"studentId": "student123", "courseId": "course1"}, label="Enrollment Lookup")

# 3. Assignment due next week
now = datetime.utcnow()
next_week = now + timedelta(days=7)
q3_before = time_query(db, "assignments", {"dueDate": {"$gte": now, "$lte": next_week}}, label="Upcoming Assignments")

In [168]:
#Creating Index
db.courses.create_index([("category", 1)])
db.enrollments.create_index([("studentId", 1), ("courseId", 1)])
db.assignments.create_index([("dueDate", 1)])


'dueDate_1'

In [169]:
# After Indexing
q1_after = time_query(db, "courses", {"category": "Programming"}, label="Course by Category")
q2_after = time_query(db, "enrollments", {"studentId": "student123", "courseId": "course1"}, label="Enrollment Lookup")
q3_after = time_query(db, "assignments", {"dueDate": {"$gte": now, "$lte": next_week}}, label="Upcoming Assignments")


**Document the performance improvements using Python timing functions**

In [170]:
print_comparison(q1_before, q1_after)
print_comparison(q2_before, q2_after)
print_comparison(q3_before, q3_after)



📊 Course by Category
  - Before Index: 5.25 ms
  - After Index:  0.93 ms
  - Improvement:  4.32 ms

📊 Enrollment Lookup
  - Before Index: 1.02 ms
  - After Index:  0.57 ms
  - Improvement:  0.45 ms

📊 Upcoming Assignments
  - Before Index: 0.53 ms
  - After Index:  0.54 ms
  - Improvement:  -0.01 ms
